In [2]:
using Pkg
Pkg.add("Distributions")
Pkg.add("Mosek")
Pkg.add("JuMP")
Pkg.add("LinearAlgebra")
Pkg.add("MultivariatePolynomials")
Pkg.add("PolyJuMP")
Pkg.add("SumOfSquares")
Pkg.add("DynamicPolynomials")
Pkg.build("Mosek")
Pkg.add("DataStructures")
Pkg.add("Optim")
using Distributions
using LinearAlgebra
using MultivariatePolynomials
using JuMP
using PolyJuMP
using SumOfSquares
using DynamicPolynomials
using Mosek
using DataStructures
using Optim

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.ju

┌ Info: Recompiling stale cache file /Users/elviramorenof/.julia/compiled/v1.0/Mosek/vS2ln.ji for Mosek [6405355b-0ac2-5fba-af84-adbd65488c0e]
└ @ Base loading.jl:1187


**Norm approximation:** 

$
\begin{equation}
\begin{aligned}
   \underset{f\in H_{n,d}}{min} & \quad \sum_{i=1}^N | ||x_i||^d-f(x_i)|\\
    \text{s.t.} \;\; & (f(x)-c(\sum_i{x_i^{2}})^{d/2})(\sum_i{x_i^{2}})^{r} \text{ sos.}\\
    & c\geq0
\label{norm regression}
\end{aligned}
\end{equation}$

In [3]:
### Norma 1 en R^2

#p-Norm we want to approximate
norma_p=1
#Number of variables (of the polynomial)
n=2
#1/2 del grado del polinomio f
d=3

#We generate a sample on the unitary ball
N=200
u=rand(Uniform(-1/(n^(1/norma_p)),1/(n^(1/norma_p))),N,n)

#We calculate the norm of the points on the sample
normas=[]
for i=1:N
    push!(normas,norm(u[i,:],1))
end

#We generate the variables x_1,...,x_n and the vector of monomials of degree 2d 
@polyvar x[1:n]
M=monomials(x,2*d)

#Evaluation in the monomials
Ev= zeros(N,length(M))
for j in 1:N
    Ev[j,:] = [subs(M[i],x => u[j,:]) for i in 1:length(M)]
end

In [4]:
####################### Optimization model #######################

modelo = SOSModel(solver = MosekSolver())

#We fix r
r=1

#Coefficients of the polynomial we are looking for (model variables).
@variable(modelo,a[1:length(M)])
@variable(modelo, c)

p = sum([a[i]*M[i] for i in 1:length(M)])

#p must be convex and positive semidefinite
s=sum(map(z->z^2,x))
q=p-c*s^d
H=differentiate(q,x,2)

@polyvar y[1:n]

z=y'*H*y
g=z*(sum(map(z->z^2,x))^r)

@constraint modelo g>=0
@constraint modelo c>=0

#Evaluation in P
pEvaluado= Ev*a

#Quantity whose absolute value we want minimized
L=pEvaluado-(map(x->x^(2*d),normas))

@variable(modelo,z[1:N])
for i in 1:N
    @constraint modelo L[i]<=z[i]
    @constraint modelo -L[i]<=z[i]
end

#Objective Function
@objective(modelo,Min,sum(z))

status =  solve(modelo)

as = getvalue(a)
FO = getvalue(sum(z))

sol = as'*M


MOSEK warning 705: #7 (nearly) zero elements are specified in sparse row ''(110) of matrix 'A'.
MOSEK warning 705: #7 (nearly) zero elements are specified in sparse row ''(111) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(276) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(277) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(402) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(403) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(404) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(405) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(446) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(447) of matrix 'A'.
Warning number 705 is disabled.
Problem


2.000146393069711x₁⁶ - 0.0004767467944670933x₁⁵x₂ + 30.000308366638606x₁⁴x₂² + 5.9459235974961665e-5x₁³x₂³ + 29.99955591972043x₁²x₂⁴ + 0.0005126461161651118x₁x₂⁵ + 1.9998520936450002x₂⁶

In [4]:

function Optimizar(r)     

modelo = SOSModel(solver = MosekSolver())


@variable(modelo,a[1:length(M)])
@variable(modelo, c)

p = sum([a[i]*M[i] for i in 1:length(M)])

s=sum(map(z->z^2,x))
q=p-c*s^d
H=differentiate(q,x,2)

@polyvar y[1:n]

z=y'*H*y
g=z*(sum(map(z->z^2,x))^r)

@constraint modelo g>=0
@constraint modelo c>=0

pEvaluado= Ev*a

L=pEvaluado-(map(x->x^(2*d),normas))

@variable(modelo,z[1:N])
for i in 1:N
    @constraint modelo L[i]<=z[i]
    @constraint modelo -L[i]<=z[i]
end

@objective(modelo,Min,sum(z))

status =  solve(modelo)

FO = getvalue(sum(z))

return(FO) 
    
end    

Optimizar (generic function with 1 method)

In [5]:
#We run the optimization model for different values of r

fo=[]
for r=0:5
    err=Optimizar(r)
    push!(fo,err)
end

print(fo)


MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(84) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(85) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(144) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(145) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(238) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(239) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(266) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(267) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(310) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(311) of matrix 'A'.
Warning number 705 is disabled.
Problem
  

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(102) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(103) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(162) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(163) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(256) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(257) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(284) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(285) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(328) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(329) of matrix 'A'.
Warning number 705 is disabled.
Problem


6   2.2e-02  1.2e-02  1.0e-01  1.03e+00   2.894825967e+00   2.901384335e+00   1.2e-02  0.01  
7   9.8e-03  5.4e-03  6.9e-02  1.01e+00   3.062870222e+00   3.065542925e+00   5.4e-03  0.01  
8   7.2e-03  4.0e-03  5.9e-02  9.51e-01   3.087620047e+00   3.089661393e+00   4.0e-03  0.01  
9   6.6e-03  3.6e-03  5.5e-02  5.10e-01   3.082950292e+00   3.084850414e+00   3.6e-03  0.01  
10  5.6e-03  3.1e-03  4.7e-02  5.43e-01   3.063904505e+00   3.065883043e+00   3.1e-03  0.01  
11  5.0e-03  2.7e-03  3.7e-02  2.83e-01   3.016272162e+00   3.019097202e+00   2.7e-03  0.01  
12  3.2e-03  1.7e-03  2.0e-02  -6.88e-02  2.859017600e+00   2.864052216e+00   1.7e-03  0.01  
13  2.5e-03  1.4e-03  1.6e-02  -2.29e-01  2.772084919e+00   2.777293800e+00   1.4e-03  0.01  
14  1.3e-03  7.1e-04  7.4e-03  -3.08e-01  2.435358864e+00   2.442545968e+00   7.1e-04  0.02  
15  1.1e-03  5.8e-04  6.5e-03  -8.97e-02  2.258199506e+00   2.264289629e+00   5.8e-04  0.02  
16  4.5e-04  2.5e-04  3.1e-03  -1.34e-01  1.494105567e+00   

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212
┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


...

28  6.3e-08  3.4e-08  1.5e-05  1.00e+00   1.379294294e-01   1.379343542e-01   3.4e-08  0.05  
29  3.3e-08  1.8e-08  1.1e-05  1.00e+00   1.371987761e-01   1.372014630e-01   1.8e-08  0.05  
30  1.8e-08  9.6e-09  7.7e-06  1.00e+00   1.368054023e-01   1.368068579e-01   9.6e-09  0.05  
31  9.7e-09  5.3e-09  5.7e-06  1.00e+00   1.366022308e-01   1.366030512e-01   5.3e-09  0.05  
32  4.8e-09  2.7e-09  3.9e-06  1.00e+00   1.364833420e-01   1.364837590e-01   2.7e-09  0.06  
33  3.6e-09  2.0e-09  3.4e-06  9.99e-01   1.364525703e-01   1.364528827e-01   2.0e-09  0.06  
34  3.6e-09  2.0e-09  3.4e-06  9.96e-01   1.364522829e-01   1.364525943e-01   2.0e-09  0.07  
35  3.6e-09  2.0e-09  3.4e-06  9.98e-01   1.364522470e-01   1.364525583e-01   2.0e-09  0.07  
36  3.6e-09  2.0e-09  3.4e-06  9.96e-01   1.364522381e-01   1.364525493e-01   2.0e-09  0.08  
37  3.6e-09  2.0e-09  3.4e-06  9.97e-01   1.364522381e-01   1.364525493e-01   2.0e-09  0.09  
Optimizer terminated. Time: 0.09    


Interior-point soluti

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


32  2.3e-09  1.3e-09  1.5e-06  1.00e+00   1.365520006e-01   1.365526787e-01   1.3e-09  0.05  
33  2.0e-09  1.1e-09  1.4e-06  9.99e-01   1.365240027e-01   1.365245812e-01   1.1e-09  0.06  
34  1.9e-09  1.0e-09  1.3e-06  9.98e-01   1.365120511e-01   1.365125868e-01   1.0e-09  0.07  
35  1.9e-09  1.0e-09  1.3e-06  9.98e-01   1.365120511e-01   1.365125868e-01   1.0e-09  0.07  
Optimizer terminated. Time: 0.08    


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 1.3651205105e-01    nrm: 1e+03    Viol.  con: 2e-07    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 1.3651258684e-01    nrm: 6e+01    Viol.  con: 2e-08    var: 2e-08    barvar: 2e-08  


┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


Any[0.136377, 0.13638, 0.136385, 0.136403, 0.136452, 0.136512]

In [6]:
#We generate the sample of the norm we want to approximate 

    norma_p=1
    n=2

    N=200
    u=rand(Uniform(-1/(n^(1/norma_p)),1/(n^(1/norma_p))),N,n)

    normas=[]
    for i=1:N
        push!(normas,norm(u[i,:],1))
    end

    r=1

#We run the optimization model for different values of d

fo=[]
for d=1:6

   
    @polyvar x[1:n]
   
    M=monomials(x,2*d)

   
    Ev= zeros(N,length(M))
    for j in 1:N
        Ev[j,:] = [subs(M[i],x => u[j,:]) for i in 1:length(M)]
    end
    
    err=Optimizar(r)
    push!(fo,err)
    
end

print(fo)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 456             
  Cones                  : 0               
  Scalar variables       : 204             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 199
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective 

6   1.0e-01  5.8e-02  1.8e-01  8.28e-01   6.626161156e+00   6.685550416e+00   5.8e-02  0.01  
7   6.7e-02  3.7e-02  1.4e-01  9.07e-01   5.540947576e+00   5.578656525e+00   3.7e-02  0.01  
8   5.5e-02  3.1e-02  1.2e-01  4.96e-01   4.935029889e+00   4.968943822e+00   3.1e-02  0.01  
9   4.0e-02  2.2e-02  1.1e-01  7.85e-01   4.324977311e+00   4.345963827e+00   2.2e-02  0.01  
10  3.0e-02  1.7e-02  9.3e-02  7.16e-01   3.823235143e+00   3.838864645e+00   1.7e-02  0.01  
11  1.5e-02  8.5e-03  6.8e-02  8.63e-01   3.100537379e+00   3.107218756e+00   8.5e-03  0.01  
12  6.1e-03  3.4e-03  4.7e-02  9.35e-01   2.644851182e+00   2.646589110e+00   3.4e-03  0.01  
13  3.1e-03  1.7e-03  3.1e-02  8.36e-01   2.436530716e+00   2.437663066e+00   1.7e-03  0.01  
14  2.4e-03  1.3e-03  2.6e-02  7.24e-01   2.380215274e+00   2.381172159e+00   1.3e-03  0.02  
15  1.9e-03  1.1e-03  2.3e-02  6.84e-01   2.338286209e+00   2.339106887e+00   1.1e-03  0.02  
16  1.3e-03  7.1e-04  1.8e-02  7.57e-01   2.276087382e+00   

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212




Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 2.1267277219e+00    nrm: 2e+02    Viol.  con: 2e-07    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 2.1267277224e+00    nrm: 1e+01    Viol.  con: 1e-09    var: 1e-09    barvar: 1e-09  
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(70) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(71) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(98) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(99) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(110) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(111) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(124) of matrix 'A'.
MOSEK warning 705: #6 (

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212
┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212


MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(60) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(61) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(66) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(67) of matrix 'A'.
MOSEK warning 705: #10 (nearly) zero elements are specified in sparse row ''(70) of matrix 'A'.
MOSEK warning 705: #10 (nearly) zero elements are specified in sparse row ''(71) of matrix 'A'.
MOSEK warning 705: #13 (nearly) zero elements are specified in sparse row ''(72) of matrix 'A'.
MOSEK warning 705: #13 (nearly) zero elements are specified in sparse row ''(73) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(98) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(99) of matrix 'A'.
Warning number 705 is disabled.
Problem
  Name

┌ Warning: Not solved to optimality, status: Stall
└ @ JuMP /Users/elviramorenof/.julia/packages/JuMP/Xvn0n/src/solvers.jl:212
